In [12]:
import numpy as np
import cv2
from matplotlib import pyplot as plt


# Extracting the Features

In [13]:
img1 = cv2.imread('images/Fifty_thousand_Dram_2.jpg',0)          # queryImage
#img2 = cv2.imread('images/Ten_thousand_Dram_3.jpg',0) # trainImage

# Initiate SIFT detector
sift = cv2.ORB_create()

# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
bf = cv2.BFMatcher()

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    kp2, des2 = sift.detectAndCompute(grayframe,None)

    # BFMatcher with default params
   
    matches = bf.knnMatch(des1,des2, k=2)
    
    # Apply ratio test.
    
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
    
    # cv2.drawMatchesKnn expects list of lists as matches.
    #img3 = cv2.drawMatchesKnn(img1,kp1,grayframe,kp2,good,None,flags=2)
    #Homograpy
    
    if len(good) > 10:
        query_pts = np.float32([kp2[i].pt for i in good]).reshape(-1, 1, 2)
        train_pts = np.float32([kp2[j].pt for j in good]).reshape(-1, 1, 2)
        
        
        matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
        matches_mask = mask.ravel().tolist()
        
        
        # Perspective transform
        h, w = img.shape
        pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, matrix)

    else:   
        cv2.imshow('Homo',grayframe) 
        
    #cv2.imshow('Frame', img3)
    
    #cv2.imshow('Frame', grayframe)
#    
    
    key = cv2.waitKey(1)
    if key == 27:
        break
    
    
cap.release()
cv2.destroyAllWindows()

In [11]:
cv2.__version__

'4.1.0'

In [6]:
cv2.line(img1,(0,0),(511,511),(255,0,0),5)


array([[255, 255, 255, ..., 253, 255, 255],
       [255, 255, 255, ..., 169, 187, 239],
       [255, 255, 255, ..., 162, 167, 252],
       ...,
       [251, 219, 186, ..., 190, 212, 251],
       [248, 226, 203, ..., 208, 218, 244],
       [255, 253, 248, ..., 247, 244, 255]], dtype=uint8)

In [41]:
query_pts = np.float32([kp2[m].pt for m in good]).reshape(-1, 1, 2)

TypeError: list indices must be integers or slices, not list